# Ant Colony

In [1]:
import numpy as np
from itertools import permutations,islice
import random
import pandas as pd
import copy

In [2]:
class City:
  def __init__(self,city,x,y):
    self.city = city
    self.x = x
    self.y = y
  def __repr__(self) -> str:
      return f'(id:{self.city})'

  def __eq__(self, other):
      return (self.city == other.city)

In [3]:
class Chromosome:
  def __init__(self,c_list,fitness=0, cost=0):
    self.c_list = c_list
    self.fitness = fitness
    self.cost = cost
  def __repr__(self) -> str:
      return f'({self.c_list}, {self.cost})'

In [4]:
data_df = pd.read_csv('15-Points.csv')

In [5]:
def calculate_distance(point1,point2):
    return np.sqrt((point1.x-point2.x)**2+(point1.y-point2.y)**2)

In [6]:
def generate_distance_matrix(chromosome):
    distance_matrix = np.zeros((len(chromosome.c_list),len(chromosome.c_list)))
    for i in range(len(chromosome.c_list)):
        for j in range(i,len(chromosome.c_list)):
            dist = calculate_distance(chromosome.c_list[i],chromosome.c_list[j])
            distance_matrix[i][j] = distance_matrix[j][i] = dist

        distance_matrix[i][i] = np.inf
    distance_matrix_dict = dict(zip([i.city for i in chromosome.c_list], distance_matrix.tolist()))

    return copy.deepcopy(distance_matrix_dict)
# generate_distance_matrix(generate_intial_population(3,data_df)[0])

In [7]:
def calculate_cost(c_list,distance_matrix_dict):
  cost=0
  for i in range(1,len(c_list)):
      cost += distance_matrix_dict[c_list[i-1].city][c_list[i].city-1]
  cost += distance_matrix_dict[c_list[0].city][c_list[-1].city-1]
  return cost

In [8]:
def calculate_fitness(cost):
    return 1/cost

In [9]:
def calculate_phermone_matrix(old_matrix, pop, p):
    new_matrix = copy.deepcopy(old_matrix) * (1-p)
    for chromosome in pop:
        fitness_val = chromosome.fitness
        for i in range(1, len(chromosome.c_list)):
            city1 = chromosome.c_list[i-1].city - 1
            city2 = chromosome.c_list[i].city - 1

            new_matrix[city1, city2] =+ fitness_val
            new_matrix[city2, city1] =+ fitness_val
            
    return new_matrix

In [10]:
def generate_intial_population(popuation_size,intial_chromosome,distance_matrix_dict):
  pop = []
  for i in range(popuation_size):
      c_list = np.random.permutation(intial_chromosome.c_list)
      cost = calculate_cost(c_list,distance_matrix_dict)
      pop.append(Chromosome(c_list,fitness=calculate_fitness(cost),cost=cost))

  return copy.deepcopy(np.array(pop))
# generate_intial_population(3,data_df)[0].c_list[0].city

In [11]:
def elitism(old_pop,elit_size):
  
    sorted_pop = copy.deepcopy(np.array(sorted(old_pop, key=lambda chrom: chrom.cost)))

    return sorted_pop[:elit_size]

In [12]:
def construction_phase(list_c, phermone_matrix, distance_matrix, alpha, beta, ants_num):
    new_generation = []
    for ant in range(ants_num):
        current_city = np.random.choice(list_c)
        path = [current_city]
        unvisited_cities = copy.deepcopy(list_c)

        for i in range(len(list_c)-1):
            unvisited_cities.remove(current_city)
            probabilities = []
            for city in unvisited_cities:

                probability = (((phermone_matrix[current_city.city-1,city.city-1])**alpha) * ((1/(distance_matrix[current_city.city][city.city-1]))**beta))
                probabilities.append(probability)
            
            probabilities = np.array(probabilities) / np.sum(probabilities)

            next_city = np.random.choice(unvisited_cities, p=probabilities)

            path.append(next_city)

            current_city = next_city

        chromosome = Chromosome(path)
        chromosome.cost = calculate_cost(chromosome.c_list, distance_matrix)
        chromosome.fitness = calculate_fitness(chromosome.cost)

        new_generation.append(chromosome)
    return new_generation
            

In [13]:
def AntColony_process(data_df,popuation_size,elit_size,iterations, alpha, beta, p):
    

    intial_chromosome = Chromosome(c_list=[City(int(row[1][2]),row[1][0],row[1][1]) for row in data_df.iterrows()]) 
    distance_matrix = generate_distance_matrix(intial_chromosome)
    pop = generate_intial_population(popuation_size,intial_chromosome,distance_matrix)
    intial_phermone_matrix = np.zeros((15,15)) + 0.0000001
    phermone_matrix = calculate_phermone_matrix(intial_phermone_matrix, pop, p)


    for i in range(iterations):

        elitism_pop = elitism(pop,elit_size)

        new_pop_size = popuation_size - elit_size

        new_pop = construction_phase(intial_chromosome.c_list, phermone_matrix, distance_matrix, alpha, beta, ants_num = new_pop_size)

        pop = np.array(copy.deepcopy(elitism_pop.tolist()) + copy.deepcopy(new_pop))

        phermone_matrix = calculate_phermone_matrix(phermone_matrix, pop, p)

    return list(sorted(pop, key=lambda chrom: chrom.fitness, reverse=True))[0].cost

In [16]:
AntColony_process(data_df,popuation_size=200,elit_size=20,iterations=40, alpha=1, beta=2, p=0.7)

284.3810904080332